In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 1000)

In [ ]:
seasons = [2015, 2016, 2017, 2018, 2019]

In [ ]:
coop_lst = []
url = 'https://raw.githubusercontent.com/ryurko/nflscrapR-data/master/play_by_play_data/regular_season/reg_pbp_'
for season in seasons:
    print(f'Reading in Play-By-Play data for the {season} season')
    season_pbp = pd.read_csv(f'{url}{season}.csv')
    coop_pbp = season_pbp.loc[season_pbp.receiver_player_id == '00-0031544']
    coop_lst.append(coop_pbp)
    
coop_df = pd.concat(coop_lst).reset_index(drop=True)

In [ ]:
#dal_pbp_df.loc[dal_pbp_df.receiver_player_name == 'A.Cooper'].receiver_player_id.value_counts()

In [ ]:
coop_df['game_date'] = pd.to_datetime(coop_df['game_date'])

In [ ]:
dal_pbp_df.sort_values('game_date')

In [ ]:
coop_df = dal_pbp_df.loc[(dal_pbp_df.play_type == 'pass') &
                         ( dal_pbp_df.receiver_player_id == '00-0031544')].reset_index(drop=True)

In [ ]:
coop_df.sort_values('game_date')

In [ ]:
coop_df['month'] = coop_df.game_date.dt.month


In [ ]:
coop_df.groupby['month']['']

In [ ]:
yards_in_a_month = coop_df.groupby(['month'], as_index = False)['yards_gained'].sum()

In [ ]:
team_yards_in_a_month = coop_df.groupby(['month', 'posteam'], as_index = False)['yards_gained'].sum() \
    .sort_values(['month', 'posteam'])

In [ ]:
sns.barplot(x= 'month', y = 'yards_gained', data=team_yards_in_a_month, hue='posteam')

In [ ]:
team_games_in_a_month = coop_df.groupby(['month', 'posteam'], as_index=False)['game_id'] \
    .agg({'games': pd.Series.nunique}) \
    .sort_values(['month', 'posteam'])

In [ ]:
graph_df = team_yards_in_a_month.merge(team_games_in_a_month, on = ['month', 'posteam'])

In [ ]:
graph_df['yards_per_game'] = graph_df['yards_gained'] / graph_df['games']

In [ ]:
sns.barplot(x='month', y='yards_per_game', hue='posteam', data=graph_df.loc[graph_df.month != 1])

In [ ]:
import calmap

In [ ]:
plt.figure(figsize=(12,18))
calmap.yearplot(coop_df.groupby('game_date')['yards_gained'].sum())

In [ ]:
coop_df.head()

In [ ]:
team_yards_in_a_qtr = coop_df.groupby(['qtr', 'posteam', 'month'], as_index = False)['yards_gained'].sum() \
    .sort_values(['qtr', 'posteam'])

In [ ]:
team_yards_in_a_qtr.head()

In [ ]:
# sns.barplot(x='qtr', y='yards_gained', data=team_yards_in_a_qtr.loc[team_yards_in_a_qtr.qtr != 5], #hue='posteam'
#            )

In [ ]:
# g = sns.FacetGrid(team_yards_in_a_qtr, col="qtr", col_wrap=2, height=1.5)
# g = g.map(plt.hist, "month", "yards_gained", marker=".")

In [ ]:
for c in coop_df.columns:
    print(c)

In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [8]:
seasons = ['2015','2016','2017','2018','2019',]
weeks = [str(w) for w in range(1, 18)]
url = 'http://www.nflweather.com/en/week/'

In [30]:
def request_data(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    return soup

def get_team_data(bs_data):
    teams = bs_data.find_all(class_="team-name text-center")

    away_teams = []
    home_teams = []

    for i, v in enumerate(teams):
        if i % 2 == 0:
            away_teams.append(teams[i].find('a').contents[0])
        else:
            home_teams.append(teams[i].find('a').contents[0])
            
    return away_teams, home_teams

def get_weather_data(bs_data):
    games = bs_data.find_all('tr')

    weather_lst = []
    for i, v in enumerate(games):
        #skip header
        if i > 0:
            game_details = games[i].find_all(class_='text-center')

            forecast = game_details[5].contents[0]

            weather_lst.append(' '.join(forecast.split()))
            
    return weather_lst

In [35]:
weather_lst = []

for season in seasons:
    for week in weeks:
        print(f'Scraping Weather Data for the {season} season week {week}')
        request_url = f'{url}{season}/week-{week}'
        soup = request_data(request_url)
        
        away_teams, home_teams = get_team_data(soup)
        
        weather = get_weather_data(soup)
        
        weather_df = pd.DataFrame({'away_tm': away_teams,
                                   'home_tm': home_teams,
                                   'weather': weather,
                                  })
        
        weather_df['season'] = season
        weather_df['week'] = week
        
        weather_lst.append(weather_df)
        
full_weather_df = pd.concat(weather_lst).reset_index(drop=True)

Requesting Data for the 2015 season week 1
Requesting Data for the 2015 season week 2
Requesting Data for the 2015 season week 3
Requesting Data for the 2015 season week 4
Requesting Data for the 2015 season week 5
Requesting Data for the 2015 season week 6
Requesting Data for the 2015 season week 7
Requesting Data for the 2015 season week 8
Requesting Data for the 2015 season week 9
Requesting Data for the 2015 season week 10
Requesting Data for the 2015 season week 11
Requesting Data for the 2015 season week 12
Requesting Data for the 2015 season week 13
Requesting Data for the 2015 season week 14
Requesting Data for the 2015 season week 15
Requesting Data for the 2015 season week 16
Requesting Data for the 2015 season week 17
Requesting Data for the 2016 season week 1
Requesting Data for the 2016 season week 2
Requesting Data for the 2016 season week 3
Requesting Data for the 2016 season week 4
Requesting Data for the 2016 season week 5
Requesting Data for the 2016 season week 6
Req

In [36]:
full_weather_df

,away_tm,home_tm,weather,season,week
0,Steelers,Patriots,64f Light Rain,2015,1
1,Packers,Bears,68f Partly Cloudy,2015,1
2,Chiefs,Texans,83f Clear,2015,1
3,Browns,Jets,76f Partly Cloudy,2015,1
4,Colts,Bills,53f Mostly Cloudy,2015,1
...,...,...,...,...,...
1275,Titans,Texans,64f Partly Cloudy,2019,17
1276,Colts,Jaguars,73f Mostly Cloudy,2019,17
1277,Raiders,Broncos,32f Clear,2019,17
1278,Cardinals,Rams,61f Clear,2019,17
